In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from enum import Enum, auto

class Player(Enum):
    MAX = auto()
    MIN = auto()
    
    def toggle(self):
        if self == Player.MAX:
            return Player.MIN
        else:
            return Player.MAX


In [5]:
class Action:
    pass

class PlaceFence(Action):
    
    def __init__(self, isHorizontal: bool, coord: (int, int)):
        self.isHorizontal = isHorizontal
        self.coord = coord
        
    def __eq__(self, other):
        if isinstance(other, PlaceFence):
            return self.isHorizontal == other.isHorizontal and self.coord == other.coord
        return False
    
    def __hash__(self):
        return hash((self.isHorizontal, self.coord))

class Move(Action):
    
    def __init__(self, from_coord: (int, int), to_coord: (int, int)):
        self.from_coord = from_coord
        self.to_coord = to_coord
        
    def __eq__(self, other):
        if isinstance(other, Move):
            return self.from_coord == other.from_coord and self.to_coord == other.to_coord
        return False
    
    def __hash__(self):
        return hash((self.from_coord, self.to_coord))

In [10]:
from copy import deepcopy

class State:
    
    def get_player(self) -> Player:
        pass

    def get_applicable_actions(self) -> list[Action]:
        pass

    def get_action_result(self, action: Action):
        pass
    
    def __eq__(self, other):
        pass
    
    def __hash__(self):
        pass

class Board(State):
    
    def __init__(
        self,
        grid_size = 9,
        player_positions: dict = None,
        current_player: Player = Player.MAX,
        fences_horizontal = set(),
        fences_vertical = set()
    ):
        self.grid_size = grid_size
        mid_point = grid_size // 2  
        if player_positions is None:
            self.player_positions = {
                Player.MAX: (mid_point, 0),            # Player.MAX starts at the top row
                Player.MIN: (mid_point, grid_size-1)   # Player.MIN starts at the bottom row
            }
        else:
            self.player_positions = player_positions
            
        self.fences_horizontal = fences_horizontal
        self.fences_vertical = fences_vertical

    def get_player(self) -> Player:
        return self.current_player
    
    def toggle_player(self):
        self.current_player = self.current_player.toggle()
    
    def get_applicable_actions(self) -> list[Action]:
        actions = []
        # Calculate possible moves for the pawn
        current_pos = self.player_positions[self.get_player()]
        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # down, right, up, left
            new_pos = (current_pos[0] + direction[0], current_pos[1] + direction[1])
            if self.is_move_valid(current_pos, new_pos):
                actions.append(Move(current_pos, new_pos))
        
        # Calculate possible fence placements
        for i in range(self.grid_size - 1):
            for j in range(self.grid_size - 1):
                if self.can_place_fence((i, j), True):
                    actions.append(PlaceFence(True, (i, j)))
                if self.can_place_fence((i, j), False):
                    actions.append(PlaceFence(False, (i, j)))

        return actions

    def get_action_result(self, action: Action):
        new_board = deepcopy(self)  # Create a deep copy of the current board
        if isinstance(action, Move):
            # Assume it's the current player's move
            new_board.player_positions[new_board.get_player()] = action.to_coord
        elif isinstance(action, PlaceFence):
            new_board.fences.add((action.isHorizontal, action.coord))
        
        new_board.toggle_player()  # Switch to the other player
        
        return new_board
    
    def is_move_valid(self, from_coord, to_coord):
        # Add logic to check if the move is valid (e.g., not blocked by a fence).
        return True

    def can_place_fence(self, coord, is_horizontal):
        # Add logic to check if a fence can be placed (not already placed, not blocking a path).
        self.fences
        return True
    
    def __eq__(self, other):
        if isinstance(other, Board):
            return (self.player_positions == other.player_positions and
                    self.fences == other.fences and
                    self.current_player == other.current_player)
        return False
    
    def __hash__(self):
        return hash((frozenset(self.player_positions.items()), frozenset(self.fences), self.current_player))


In [9]:
class FenceManager:
    def __init__(self, grid_size, fence_length):
        self.grid_size = grid_size
        self.fence_length = fence_length

    def can_place_fence(self, coord, is_horizontal):
        # Check if the fence is within the bounds of the board
        if is_horizontal:
            if coord[0] > self.grid_size - self.fence_length or coord[1] >= self.grid_size:
                return False
        else:
            if coord[1] > self.grid_size - self.fence_length or coord[0] >= self.grid_size:
                return False

        # Check for overlapping fences
        if is_horizontal:
            for i in range(self.fence_length):
                if (coord[0] + i, coord[1]) in self.fences_horizontal:
                    return False
            # Check if it's connecting with another horizontal fence
            if (coord[0] - 1, coord[1]) in self.fences_horizontal or (coord[0] + self.fence_length, coord[1]) in self.fences_horizontal:
                return False
        else:
            for i in range(self.fence_length):
                if (coord[0], coord[1] + i) in self.fences_vertical:
                    return False
            # Check if it's connecting with another vertical fence
            if (coord[0], coord[1] - 1) in self.fences_vertical or (coord[0], coord[1] + self.fence_length) in self.fences_vertical:
                return False

        # Check if the new fence will not intersect with existing vertical fences
        if is_horizontal:
            for i in range(self.fence_length):
                if (coord[0] + i, coord[1]) in self.fences_vertical:
                    return False
        else:
            for i in range(self.fence_length):
                if (coord[0], coord[1] + i) in self.fences_horizontal:
                    return False

        # Check that the fence does not block all paths to the goal
        # This requires a pathfinding algorithm to ensure that both players still have a path to win.
        # ...

        return True

    def place_fence(self, coord, is_horizontal):
        if self.can_place_fence(coord, is_horizontal):
            if is_horizontal:
                for i in range(self.fence_length):
                    self.fences_horizontal.add((coord[0] + i, coord[1]))
            else:
                for i in range(self.fence_length):
                    self.fences_vertical.add((coord[0], coord[1] + i))
            return True
        else:
            return False


In [ ]:
from collections import deque

class QuoridorPathfinder:
    def __init__(self, fences_horizontal, fences_vertical, grid_size):
        self.fences_horizontal = fences_horizontal
        self.fences_vertical = fences_vertical
        self.grid_size = grid_size

    def is_path_blocked(self, from_coord, to_coord, is_horizontal):
        if is_horizontal:
            return (from_coord, to_coord) in self.fences_horizontal or (from_coord[0], from_coord[1] - 1) in self.fences_horizontal
        else:
            return (from_coord, to_coord) in self.fences_vertical or (from_coord[0] - 1, from_coord[1]) in self.fences_vertical

    def get_neighbors(self, position):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Down, Right, Up, Left
        neighbors = []
        for d in directions:
            neighbor = (position[0] + d[0], position[1] + d[1])
            if 0 <= neighbor[0] < self.grid_size and 0 <= neighbor[1] < self.grid_size and not self.is_path_blocked(position, neighbor, d[1] == 0):
                neighbors.append(neighbor)
        return neighbors

    def bfs(self, start, goal_line):
        queue = deque([start])
        visited = set([start])

        while queue:
            current = queue.popleft()
            if current[1] == goal_line:
                return True  # Reached the goal line

            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)

        return False  # No path found

    def path_exists_for_both_players(self, player_positions):
        # Player.MAX tries to reach the bottom row (grid_size - 1), and Player.MIN tries to reach the top row (0)
        return self.bfs(player_positions[Player.MAX], self.grid_size - 1) and self.bfs(player_positions[Player.MIN], 0)


: 

In [10]:
from UIKit import UIBoard
ui_board = UIBoard(fences_horizontal=[(4, 0)], fences_vertical=[(5, 0)])
ui_board.draw()

.   .   .   .   ■   . | .   .   .   
                —————                 
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   .   .   .   .   .   
                                    
.   .   .   .   ●   .   .   .   .   
                                    


TypeError: __init__() missing 4 required positional arguments: 'board_size', 'fences_horizontal', 'fences_vertical', and 'player_positions'